In [1]:
!export LD_LIBRARY_PATH=/opt/conda/lib
!export XLA_FLAGS=--xla_gpu_cuda_data_dir=/home/jarugula/cuda_data_dir
!export PATH=/usr/local/cuda-11.7/bin:${PATH}

In [ ]:
# Import necessary packages

import logging
import warnings
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_curve
from scipy.stats import uniform, norm
import matplotlib.pyplot as plt
import matplotlib
import random
from tqdm import tqdm
import pandas as pd

import h5py
import os
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Reshape,Conv1D,Flatten,Dense, Lambda
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  pass
from sklearn.utils import shuffle
AUTOTUNE = tf.data.AUTOTUNE
from sklearn.preprocessing import StandardScaler
import tqdm
import gc
import wandb
cols = ["#DB4437", "#4285F4", "#F4B400", "#0F9D58", "purple", "goldenrod", "peru",
        "coral", "turquoise", 'gray', 'navy', 'm', 'darkgreen', 'fuchsia', 'steelblue']

os.environ['PATH']=os.environ['PATH']+':/usr/local/cuda/bin'
os.environ['XLA_FLAGS']='--xla_gpu_cuda_data_dir=/home/jarugula/cuda_data_dir'

import sys
sys.path.insert(1, '/home/jarugula/deeplenstronomy_cosmology/deeplenstronomy')

### Data import and pre-processing

In [4]:
# The training data of 1M strong lens images
data_path = '/deepskieslab/stronglensing/hsbi/datasets/'
image_dir = 'w0_8param_fixzv_train_1M'
column_name = "w0-g" # Dark energy equation-of-state parameter 
fig_title = 'w0'

In [5]:
# Read the images and metadata
if os.path.isdir(data_path+image_dir):
    images = np.load(data_path+image_dir+'/CONFIGURATION_1_images.npy', allow_pickle=True)
    metadata = pd.read_csv(data_path+image_dir+'/CONFIGURATION_1_metadata.csv')
else:
   print("Data not found")


In [6]:
metadata.head()

,OBJID-g,H0-g,Om0-g,Ode0-g,w0-g,NAME-g,exposure_time-g,numPix-g,pixel_scale-g,psf_type-g,...,PLANE_2-OBJECT_1-LIGHT_PROFILE_1-e1-g,PLANE_2-OBJECT_1-LIGHT_PROFILE_1-e2-g,PLANE_2-OBJECT_1-MASS_PROFILE_1-NAME-g,PLANE_2-OBJECT_1-MASS_PROFILE_1-theta_E-g,PLANE_2-OBJECT_1-MASS_PROFILE_1-e1-g,PLANE_2-OBJECT_1-MASS_PROFILE_1-e2-g,PLANE_2-OBJECT_1-MASS_PROFILE_1-center_x-g,PLANE_2-OBJECT_1-MASS_PROFILE_1-center_y-g,BACKGROUND_IDX-g,PLANE_1-OBJECT_1-MASS_PROFILE_1-theta_E-g
0,0,70,0.3,0.7,-0.938552,wCDM,90.0,32,0.263,GAUSSIAN,...,0.078436,-0.082612,SIE,2.0,0.1,-0.1,0.0,0.0,0,1.059521
1,1,70,0.3,0.7,-1.299610,wCDM,90.0,32,0.263,GAUSSIAN,...,0.029977,0.008988,SIE,2.0,0.1,-0.1,0.0,0.0,0,1.063520
2,2,70,0.3,0.7,-1.634069,wCDM,90.0,32,0.263,GAUSSIAN,...,-0.031950,-0.068904,SIE,2.0,0.1,-0.1,0.0,0.0,0,1.065415
3,3,70,0.3,0.7,-0.411026,wCDM,90.0,32,0.263,GAUSSIAN,...,0.061426,0.045946,SIE,2.0,0.1,-0.1,0.0,0.0,0,1.046473
4,4,70,0.3,0.7,-1.109861,wCDM,90.0,32,0.263,GAUSSIAN,...,0.072341,0.015470,SIE,2.0,0.1,-0.1,0.0,0.0,0,1.061751


In [7]:
images = np.einsum('lkij->lijk',images)
theta = metadata[column_name].to_numpy()

In [9]:
# normalize image each image by the sum of all pixels, make it such that the sum of all pixels is 1024 (32 X32)
images = 1024*(images/np.sum(images, axis=(1,2), keepdims=True))

# manually standardies pixels across all images. 
# In this analysis we do not standerdize the images and parameter. Hence we use mean=0 and std=1.0

images = images.reshape(images.shape[0], -1)
# means_image = np.mean(images, axis=0)
# std_image = np.std(images, axis=0)
means_image = 0.0
std_image = 1.0
images = (images - means_image) / std_image
images = images.reshape(images.shape[0], 32, 32, 1)


#manually standardize the theta (w)
mean_theta = 0.0 
std_theta = 1.0 
theta = (theta - mean_theta)/std_theta

# save the mean and std for later use 
# np.savez('mean_std_normall.npz', mean_theta=mean_theta, std_theta=std_theta, means_image=means_image, std_image=std_image)

### Split the data into train and test

In [10]:
# Split into train and test
x_train , x_test, theta_train, theta_test, y_train, y_test = train_test_split(images, theta, np.ones_like(theta), test_size=0.2, random_state=0)
true_test_theta = np.copy(theta_test)
true_test_theta = true_test_theta*std_theta + mean_theta # unstanderdized parameter

del images, theta
gc.collect()

0

### Train the Neural network model 

In [13]:
seed = 128 # random weight initialization
inputs_img = keras.Input(shape=x_train[0].shape)
inputs_theta = keras.Input(shape=(1,))
custom_initializer = tf.keras.initializers.he_uniform(seed=seed)

# Entry block for ResNet
x = layers.BatchNormalization()(inputs_img)
x = layers.Conv2D(128/16, 3, strides=2, padding="same",kernel_initializer=custom_initializer)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)

previous_block_activation = x  # Set aside residual

for size in [256/16, 512/16, 728/16]:
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(size, 3, padding="same",kernel_initializer=custom_initializer)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Activation("relu")(x)
    x = layers.Conv2D(size, 3, padding="same",kernel_initializer=custom_initializer)(x)
    x = layers.BatchNormalization()(x)

    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

    # Project residual
    residual = layers.Conv2D(size, 1, strides=2, padding="same",kernel_initializer=custom_initializer)(
        previous_block_activation
    )
    x = layers.add([x, residual])  # Add back residual
    previous_block_activation = x  # Set aside next residual

x = layers.SeparableConv2D(1024/16, 3, padding="same",kernel_initializer=custom_initializer)(x)
x = layers.Activation("relu")(x)
x = layers.GlobalAveragePooling2D()(x)

# embed the theta value to the image
y = layers.Dense(64,kernel_initializer=custom_initializer)(inputs_theta)

#concatenate the theta value to the image
x = tf.concat([x, y], axis=-1)

x = layers.Dense(128, kernel_regularizer=tf.keras.regularizers.l2(0.01),kernel_initializer=custom_initializer)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)
x = layers.Dropout(0.1)(x)

x = layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.01),kernel_initializer=custom_initializer)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)
x = layers.Dropout(0.1)(x)

outputs = layers.Dense(1)(x) # Outputs the log likelihood-to-evidence ratio : log p(x|theta)/p(x)

model = keras.Model(inputs=[inputs_img, inputs_theta], outputs=outputs)
model.summary()

2024-05-19 06:40:04.905478: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-19 06:40:04.905907: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-19 06:40:04.906183: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 1)]          0         []                            
                                                                                                  
 batch_normalization (Batch  (None, 32, 32, 1)            4         ['input_1[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 conv2d (Conv2D)             (None, 16, 16, 8)            80        ['batch_normalization[0][0]'] 
                                                                                                  
 batch_normalization_1 (Bat  (None, 16, 16, 8)            32        ['conv2d[0][0]']          

In [14]:
# Custom model for training and testing
class SNREmodel(keras.Model):
    def __init__(self, model, **kwargs):
        super(SNREmodel, self).__init__(**kwargs)
        self.model = model

    def train_step(self, data):
        image, theta = data
        labels = tf.ones_like(theta) # generate class label=1 for p(x,theta)

        # shuffle the theta to generate p(x)p(theta) which is assigned a class label=0
        images_shuffle = tf.identity(image)
        theta_shuffle = tf.identity(theta)
        theta_shuffle = tf.random.shuffle(theta_shuffle)
        labels_shuffle = tf.zeros_like(theta_shuffle)

        image = tf.concat([image, images_shuffle], axis=0)
        theta = tf.concat([theta, theta_shuffle], axis=0)
        labels = tf.concat([labels, labels_shuffle], axis=0)

        index = tf.range(tf.shape(image)[0])
        index = tf.random.shuffle(index)
        image = tf.gather(image, index)
        theta = tf.gather(theta, index)
        labels = tf.gather(labels, index)

        with tf.GradientTape() as tape:
            predictions = self.model([image, theta], training=True)
            #use a binary crossentropy loss
            loss = self.compiled_loss(labels, predictions)
        
        trainable_vars = self.model.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        self.compiled_metrics.update_state(labels, predictions)
        return {m.name: m.result() for m in self.metrics}
    
    def test_step(self, data):
        image, theta = data
        labels = tf.ones_like(theta)

        images_shuffle = tf.identity(image)
        theta_shuffle = tf.identity(theta)
        theta_shuffle = tf.random.shuffle(theta_shuffle)
        labels_shuffle = tf.zeros_like(theta_shuffle)

        image = tf.concat([image, images_shuffle], axis=0)
        theta = tf.concat([theta, theta_shuffle], axis=0)
        labels = tf.concat([labels, labels_shuffle], axis=0)

        index = tf.range(tf.shape(image)[0])
        index = tf.random.shuffle(index)
        image = tf.gather(image, index)
        theta = tf.gather(theta, index)
        labels = tf.gather(labels, index)

        predictions = self.model([image, theta], training=False)
        loss = self.compiled_loss(labels, predictions)
        for metric in self.metrics:
            if metric.name == 'loss':
                metric.update_state(loss)
            else:
                metric.update_state(labels, predictions)
        return {m.name: m.result() for m in self.metrics}

In [ ]:
# Run the training
snre_model = SNREmodel(model)
snre_model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2),
              metrics=[keras.metrics.BinaryAccuracy(name="acc"), keras.metrics.AUC(name="auc")])

callbacks_ = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True), 
              tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,patience=5, min_lr=1e-6)]

history = snre_model.fit(x_train, theta_train, validation_split=0.2, epochs=100, batch_size=1024, callbacks=callbacks_)
trained_model = snre_model.model



In [16]:
model = trained_model

In [18]:
# save the model
model.save('working_model_1M-2-034_seed128_v2.keras')